# Gemini Usage

In [ ]:
# !pip install google-generativeai==0.8.3

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [ ]:
import google.generativeai as genai
import json
from utils import load_prompts
prompts = load_prompts("prompts.yaml")

In [ ]:
print(prompts)

{'system': {'general': 'You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION.'}, 'user': {'general': 'premise: {PREMISE_HERE}, hypothesis: {HYPOTHESIS_HERE}.', 'json_mode': "Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)", 'few': 'USER: premise: {PREMISE_1}, hypothesis: {HYPOTHESIS_1}.\nMODEL: {RESULT_1}\nUSER: premise: {PREMISE_2}, hypothesis: {HYPOTHESIS_2}.\nMODEL: {RESULT_2}\nUSER: premise: {PREMISE_3}, hypothesis: {HYPOTHESIS_3}.\nMODEL:'}, 'mutual': {'few_hint': 'Please first check the following examples.'}}


In [ ]:
MY_GOOGLE_API_KEY = "MY_GOOGLE_API_KEY"
genai.configure(api_key=MY_GOOGLE_API_KEY)

In [ ]:
MODEL_NAME = "gemini-1.5-pro"
TEMPERATURE = 0

## Example for Classification
- Dataset: SemEval 2014 Task1-2 (3-class classification)

In [ ]:
inputs = {
    "premise": "A group of kids is playing in a yard and an old man is standing in the background",
    "hypothesis": "A group of boys in a yard is playing and a man is standing in the background",
}

In [ ]:
system_prompt = prompts["system"]["general"]
user_prompt = prompts["user"]["general"]

In [ ]:
print(system_prompt)
print(user_prompt)

You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION.
premise: {PREMISE_HERE}, hypothesis: {HYPOTHESIS_HERE}.


In [ ]:
model = genai.GenerativeModel(
    MODEL_NAME,
    generation_config={"temperature": TEMPERATURE},
    system_instruction=system_prompt,
)

In [ ]:
cur_user_prompt = user_prompt.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.


In [ ]:
response = model.generate_content(cur_user_prompt)

In [ ]:
print(response.text)

NEUTRAL.  The premise states "kids," which could include girls. The hypothesis specifies "boys."  The premise says "old man," while the hypothesis just says "man."  While an old man *is* a man, the hypothesis doesn't require the man to be old.  Therefore, the hypothesis is neither entailed nor contradicted by the premise.



In [ ]:
num_sys_tokens = str(model.count_tokens(system_prompt)).split(": ")[1].strip()
num_usr_tokens = str(model.count_tokens(cur_user_prompt)).split(": ")[1].strip()
num_input_tokens = int(num_sys_tokens) + int(num_usr_tokens)
print(f"Num of input tokens: {num_input_tokens}")

Num of input tokens: 173


In [ ]:
num_output_tokens = str(model.count_tokens(response.text)).split(": ")[1].strip()
print(f"Num of output tokens {num_output_tokens}")

Num of output tokens 119


### Generate structured output in JSON

In [ ]:
user_prompt_json = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [ ]:
cur_user_prompt = user_prompt_json.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)


In [ ]:
response = model.generate_content(
    cur_user_prompt,
    generation_config={
        "response_mime_type": "application/json",
    },
)

In [ ]:
type(response.text)

str

In [ ]:
result_json = json.loads(response.text)
print(type(result_json))
print(result_json)

<class 'dict'>
{'result': 'NEUTRAL'}


### Few-shot

In [ ]:
fs_user_prompt = prompts["user"]["few"]

In [ ]:
cur_fs_user_prompt = fs_user_prompt.format(
    PREMISE_1="A group of kids is playing in a yard and an old man is standing in the background",
    HYPOTHESIS_1="A group of boys in a yard is playing and a man is standing in the background",
    RESULT_1="{'result': 'NEUTRAL'}",
    PREMISE_2="A man, a woman and two girls are walking on the beach",
    HYPOTHESIS_2="A group of people is on a beach",
    RESULT_2="{'result': 'ENTAILMENT'}",
    PREMISE_3="Two teams are competing in a football match",
    HYPOTHESIS_3="Two groups of people are playing football",
)
print(cur_fs_user_prompt)

USER: premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.
MODEL: {'result': 'NEUTRAL'}
USER: premise: A man, a woman and two girls are walking on the beach, hypothesis: A group of people is on a beach.
MODEL: {'result': 'ENTAILMENT'}
USER: premise: Two teams are competing in a football match, hypothesis: Two groups of people are playing football.
MODEL:


In [ ]:
fs_system_prompt = prompts["system"]["general"] + " " + prompts["mutual"]["few_hint"]
print(fs_system_prompt)

You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION. Please first check the following examples.


In [ ]:
model = genai.GenerativeModel(
    MODEL_NAME,
    generation_config={"temperature": TEMPERATURE},
    system_instruction=fs_system_prompt,
)

In [ ]:
response = model.generate_content(
    cur_fs_user_prompt,
    generation_config={
        "response_mime_type": "application/json",
    },
)

In [ ]:
result_json = json.loads(response.text)
print(result_json)

{'result': 'ENTAILMENT'}


## Example for Summarization

In [ ]:
system_prompt = """你是個中文文本摘要的專家，現在請你對一篇輸入的文章進行摘要。"""

model = genai.GenerativeModel(
    MODEL_NAME,
    generation_config={"temperature": TEMPERATURE},
    system_instruction=system_prompt,
)

In [ ]:
input_source_txt = "新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。"

In [ ]:
response = model.generate_content(input_source_txt)

In [ ]:
print(response.text)

'新修订的《中华人民共和国立法法》已正式发布，共六章105条，涵盖总则、法律、行政法规、地方性法规及规章、适用与备案审查以及附则等方面。\n'

# Claude Usage

In [ ]:
# !pip install anthropic==0.39.0

In [ ]:
import anthropic
import json
import re
from utils import load_prompts
prompts = load_prompts("prompts.yaml")

In [ ]:
MY_anthropic_KEY = "MY_anthropic_KEY"
client = anthropic.Anthropic(api_key=MY_anthropic_KEY)

In [ ]:
MODEL_NAME = "claude-3-5-sonnet-20241022"
TEMPERATURE = 0

In [ ]:
system_prompt = prompts["system"]["general"]

In [ ]:
user_prompt_json = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [ ]:
inputs = {
    "premise": "A group of kids is playing in a yard and an old man is standing in the background",
    "hypothesis": "A group of boys in a yard is playing and a man is standing in the background",
}

In [ ]:
cur_user_prompt = user_prompt_json.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)


In [ ]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    temperature=TEMPERATURE,
    system=system_prompt,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": cur_user_prompt}],
        },
    ],
)
result = response.content[0].text

In [ ]:
result

'Let me analyze this carefully:\n\n1. Premise mentions "kids" (gender-neutral) while hypothesis specifies "boys" (male)\n2. Premise mentions "old man" while hypothesis just says "man"\n3. Both texts agree on:\n   - Group playing in a yard\n   - Man standing in background\n\nSince the hypothesis specifies "boys" when the premise only mentions "kids" (which could be boys, girls, or mixed), and drops the "old" qualifier for the man, this is a NEUTRAL case. The hypothesis could be true, but it\'s not necessarily entailed by the premise.\n\n{\n    "result": "NEUTRAL"\n}'

In [ ]:
# Extract the JSON-like part of the response using regex to match a JSON object
json_match = re.search(r"\{.*?\}", response.content[0].text, re.DOTALL)

In [ ]:
json_str = json_match.group(0)  # Extract the JSON string
# Now parse the extracted JSON
result = json.loads(json_str, strict=False)

In [ ]:
result

{'result': 'NEUTRAL'}

In [ ]:
print(f"Num of input tokens {response.usage.input_tokens}")
print(f"Num of output tokens {response.usage.output_tokens}")

Num of input tokens 122
Num of output tokens 149


## Few-shot

In [ ]:
fs_system_prompt = prompts["system"]["general"] + " " + prompts["mutual"]["few_hint"]

In [ ]:
fs_user_prompt = prompts["user"]["few"]

In [ ]:
cur_fs_user_prompt = fs_user_prompt.format(
    PREMISE_1="A group of kids is playing in a yard and an old man is standing in the background",
    HYPOTHESIS_1="A group of boys in a yard is playing and a man is standing in the background",
    RESULT_1="{'result': 'NEUTRAL'}",
    PREMISE_2="A man, a woman and two girls are walking on the beach",
    HYPOTHESIS_2="A group of people is on a beach",
    RESULT_2="{'result': 'ENTAILMENT'}",
    PREMISE_3="Two teams are competing in a football match",
    HYPOTHESIS_3="Two groups of people are playing football",
)
print(cur_fs_user_prompt)

USER: premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.
MODEL: {'result': 'NEUTRAL'}
USER: premise: A man, a woman and two girls are walking on the beach, hypothesis: A group of people is on a beach.
MODEL: {'result': 'ENTAILMENT'}
USER: premise: Two teams are competing in a football match, hypothesis: Two groups of people are playing football.
MODEL:


In [ ]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    temperature=TEMPERATURE,
    system=fs_system_prompt,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": cur_user_prompt}],
        },
    ],
)
result = response.content[0].text

In [ ]:
# Extract the JSON-like part of the response using regex to match a JSON object
json_match = re.search(r"\{.*?\}", response.content[0].text, re.DOTALL)
json_str = json_match.group(0)  # Extract the JSON string
# Now parse the extracted JSON
result = json.loads(json_str, strict=False)

In [ ]:
result

{'result': 'NEUTRAL'}

# OpenAI Usage

## Single-shot

In [ ]:
!pip install openai==1.78.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.4/680.4 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [ ]:
from openai import OpenAI
import json
from utils import load_prompts
prompts = load_prompts("prompts.yaml")

In [ ]:
print(type(prompts))
print(prompts.keys())
print("Prompts 的內容: ")
for outer_k in prompts.keys():
    print(f"========== 現在 key 是 {outer_k} ==========")
    for inner_k in prompts[outer_k].keys():
        print(f"{outer_k}['{inner_k}'] = {prompts[outer_k][inner_k]}")

<class 'dict'>
dict_keys(['system', 'user', 'mutual'])
Prompts 的內容: 
========== 現在 key 是 system ==========
system['general'] = You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION.
========== 現在 key 是 user ==========
user['general'] = premise: {PREMISE_HERE}, hypothesis: {HYPOTHESIS_HERE}.
user['json_mode'] = Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)
user['few'] = USER: premise: {PREMISE_1}, hypothesis: {HYPOTHESIS_1}.
MODEL: {RESULT_1}
USER: premise: {PREMISE_2}, hypothesis: {HYPOTHESIS_2}.
MODEL: {RESULT_2}
USER: premise: {PREMISE_3}, hypothesis: {HYPOTHESIS_3}.
MODEL:
========== 現在 key 是 mutual ==========
mutual['few_hint'] = Please first check the following examples.


In [ ]:
MY_openai_KEY = "API_KEY"
client = OpenAI(api_key=MY_openai_KEY)

### 常見 OpenAI 模型與價格
- 更詳細的資訊 -> https://platform.openai.com/docs/pricing
- Price per 1M tokens

| Model                   | Input   | Cached Input | Output   |
| ----------------------- | ------- | ------------ | -------- |
| gpt-4.1                 | \$2.00  | \$0.50       | \$8.00   |
| gpt-4.1-mini            | \$0.40  | \$0.10       | \$1.60   |
| gpt-4.1-nano            | \$0.10  | \$0.025      | \$0.40   |
| gpt-4.5-preview         | \$75.00 | \$37.50      | \$150.00 |
| gpt-4o                  | \$2.50  | \$1.25       | \$10.00  |
| gpt-4o-realtime-preview | \$5.00  | \$2.50       | \$20.00  |
| gpt-4o-mini             | \$0.15  | \$0.075      | \$0.60   |


In [ ]:
MODEL_NAME = "gpt-4.1-mini"
TEMPERATURE = 0
MAX_GEN_TOKENS = 100

In [ ]:
system_prompt = prompts["system"]["general"]

In [ ]:
user_prompt_for_json = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [ ]:
inputs = {
    "premise": "A group of kids is playing in a yard and an old man is standing in the background",
    "hypothesis": "A group of boys in a yard is playing and a man is standing in the background",
}

In [ ]:
cur_user_prompt = user_prompt_for_json.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)


### JSON 生成模式 (JSON mode)
- 生成的內容結構化，容易收集特定輸出

In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": cur_user_prompt},
        # 其實是可以接著加進去更多內容
    ],
    temperature=TEMPERATURE,
    max_completion_tokens=MAX_GEN_TOKENS, # 以前這個參數叫做 `max_tokens`
)

In [ ]:
response

ChatCompletion(id='chatcmpl-BW3LzCt1vBjmuWuDMhkHCQktVGRFF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "result": "ENTAILMENT"\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1746979087, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_38647f5e19', usage=CompletionUsage(completion_tokens=12, prompt_tokens=127, total_tokens=139, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### OpenAI API Output - Organized Format

```json
{
  "id": "chatcmpl-BW3LzCt1vBjmuWuDMhkHCQktVGRFF",
  "object": "chat.completion",
  "created": 1746979087,
  "model": "gpt-4.1-mini-2025-04-14",
  "service_tier": "default",
  "system_fingerprint": "fp_38647f5e19",
  "usage": {
    "prompt_tokens": 127,
    "completion_tokens": 12,
    "total_tokens": 139,
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"result\": \"ENTAILMENT\"\n}",
        "annotations": [],
        "refusal": null,
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ]
}
```

In [ ]:
result_json = json.loads(response.choices[0].message.content)

In [ ]:
result_json

{'result': 'ENTAILMENT'}

#### 查看使用的 tokens 數量 (看自己花多少錢)
- 也可以到官方網站儀表板查看 -> https://platform.openai.com/usage

In [ ]:
print(f"Num of input tokens {response.usage.prompt_tokens}")
print(f"Num of output tokens {response.usage.completion_tokens}")

Num of input tokens 127
Num of output tokens 12


### 一般生成模式
- 生成的內容較不結構化，甚至冗長

In [ ]:
user_prompt = prompts["user"]["general"]

In [ ]:
cur_user_prompt = user_prompt.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": cur_user_prompt},
        # 其實是可以接著加進去更多內容
    ],
    temperature=TEMPERATURE,
    max_completion_tokens=MAX_GEN_TOKENS, # 以前這個參數叫做 `max_tokens`
)

In [ ]:
print(response.choices[0].message.content)

The premise states: "A group of kids is playing in a yard and an old man is standing in the background."

The hypothesis states: "A group of boys in a yard is playing and a man is standing in the background."

Analysis:
- "A group of kids" vs. "A group of boys": "Kids" is a more general term that includes boys and girls, while "boys" specifies only male children. The hypothesis narrows the group to boys only, which is not


## Few-shot

In [ ]:
fs_system_prompt = prompts["system"]["general"] + " " + prompts["mutual"]["few_hint"]

In [ ]:
# For OpenAI API, when using response_format={"type": "json_object"},
# we must tell the model to output in JSON in the user prompts.
user_prompt = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [ ]:
few_shot_inputs = [
    [
        "A group of kids is playing in a yard and an old man is standing in the background",
        "A group of boys in a yard is playing and a man is standing in the background"
    ],
    [
        "A man, a woman and two girls are walking on the beach",
        "A group of people is on a beach"
    ],
]
few_shot_labels = ["NEUTRAL", "ENTAILMENT"]

# inputs without label
real_inputs = [
        "Two teams are competing in a football match",
        "Two groups of people are playing football"
]

In [ ]:
messages = [{"role": "system", "content": fs_system_prompt}]
for sents, label in zip(few_shot_inputs, few_shot_labels):
    cur_user_prompt = user_prompt.format(
        PREMISE_HERE=sents[0],
        HYPOTHESIS_HERE=sents[1]
    )
    messages.append({"role": "user", "content": cur_user_prompt})
    messages.append({"role": "assistant", "content": "{"+"'result': "+f"{label}"+"}"})

In [ ]:
cur_user_prompt = user_prompt.format(
    PREMISE_HERE=real_inputs[0],
    HYPOTHESIS_HERE=real_inputs[1]
)
messages.append({"role": "user", "content": cur_user_prompt})

In [ ]:
messages

[{'role': 'system',
  'content': 'You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION. Please first check the following examples.'},
 {'role': 'user',
  'content': "premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)"},
 {'role': 'assistant', 'content': "{'result': NEUTRAL}"},
 {'role': 'user',
  'content': "premise: A man, a woman and two girls are walking on the beach, hypothesis: A group of people is on a beach. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)"},
 {'role': 'assistant', 'content': "{'result': ENTAILMENT}"},
 {'role': 'user',
  'conten

In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    response_format={"type": "json_object"},
    messages=messages,
    temperature=TEMPERATURE,
    max_completion_tokens=MAX_GEN_TOKENS, # 以前這個參數叫做 `max_tokens`
)
result_json = json.loads(response.choices[0].message.content)

In [ ]:
result_json

{'result': 'ENTAILMENT'}

# Google Cloud (Vertex AI) for Llama Usage

In [ ]:
!pip install openai==1.78.0

In [ ]:
import openai

## 使用 llama3-405b
```python
LOCATION = "us-central1"
MODEL_ID = "meta/llama-3.1-405b-instruct-maas"
vertex_ai_endpoint_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi/chat/completions?"
```

In [ ]:
# 設定 Google Cloud 專案資訊
PROJECT_ID = "llama3-405b-459201"
LOCATION = "us-central1" # 使用的區域
MODEL_ID = "meta/llama-3.1-405b-instruct-maas" # 確認最新的模型ID

# 設定 Vertex AI 端點 URL
# 注意：實際的端點格式可能需要查閱最新文件
vertex_ai_endpoint_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi/chat/completions?"

In [ ]:
# 先開啟 Google Cloud 終端機
# 執行 gcloud auth print-access-token 取得 api key

access_token_id = "MY_API_KEY"

In [ ]:
client = openai.OpenAI(
    base_url=f"https://us-central1-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/us-central1/endpoints/openapi/chat/completions?",
    api_key=access_token_id # 使用 gcloud access token 作為 API 金鑰
)

In [ ]:
completion = client.chat.completions.create(
    model=MODEL_ID, # 指定 Llama 模型 ID
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "請用繁體中文介紹一下台灣的玉山國家公園"}
    ],
    # stream=True # 如果需要即時生成結果
    # max_tokens=1024,
    temperature=0.0,
    # top_p=1.0,
)
print(completion.choices[0].message.content)

玉山國家公園位於台灣南部，成立於1985年4月10日，是台灣的第三個國家公園，也是台灣最高的國家公園。該公園以玉山為中心，涵蓋南投縣、嘉義縣、高雄市等地區，總面積達1,031平方公里。

玉山國家公園以其壯麗的山景、豐富的生態資源和多樣的地質景觀而聞名。其中，玉山主峰海拔3,952公尺，是台灣最高峰，也是東亞第一高峰。除了玉山主峰外，公園內還有多座高山，如玉山南峰、玉山東峰等。

公園內的生態資源非常豐富，有許多珍貴的植物和動物物種。其中，包括台灣黑熊、台灣雲豹、台灣獼猴等珍貴動物，以及玉山箭竹、玉山杜鵑等特有植物。

玉山國家公園也是台灣原住民的重要文化遺址。公園內有許多原住民部落和遺址，其中包括布農族、鄒族等原住民的傳統領域。

除了自然景觀和文化遺址外，玉山國家公園還提供許多遊憩活動，如登山、健行、賞花等。每年3月至5月，公園內的杜鵑花盛開，吸引許多遊客前來賞花。

總之，玉山國家公園是台灣最重要的自然景觀之一，也是台灣原住民的重要文化遺址。它以其壯麗的山景、豐富的生態資源和多樣的地質景觀而聞名，是台灣最值得一訪的國家公園之一。


## 使用 llama4-marverick
```python
LOCATION = "us-east5"
MODEL_ID = "meta/llama-4-maverick-17b-128e-instruct-maas"
MAAS_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
vertex_ai_endpoint_url = f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi"
```

(用法同上方 [`使用 llama3-405b`](https://colab.research.google.com/drive/1cn8yJ2fHVIoH6OHN5xDxyHy7Gv7yKexa#scrollTo=ZgjMrhssTqBj&line=6&uniqifier=1) 的範例)